<a href="https://colab.research.google.com/github/talkin24/GradientBoosting/blob/main/Ch06_XGB_Hyperparameters_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/GradientBoosting

Mounted at /content/drive
/content/drive/MyDrive/GradientBoosting


In [2]:
 import pandas as pd

 df = pd.read_csv('./heart_disease2.csv')
 df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [4]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [5]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [6]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')

In [7]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(model, X, y, cv=5)
print('accuracy:', scores)
print('accuracy mean:', scores.mean())

accuracy: [0.85245902 0.85245902 0.7704918  0.78333333 0.76666667]
accuracy mean: 0.8050819672131148


In [8]:
from sklearn.model_selection import StratifiedKFold

In [9]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [10]:
print('전체 데이터 클래스 분포:', np.bincount(y))
print()
for split_no, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
  print('{}번째 훈련 폴드:'.format(split_no+1), np.bincount(y[train_idx]))
  print('{}번째 검증 폴드:'.format(split_no+1), np.bincount(y[test_idx]))
  print()

전체 데이터 클래스 분포: [138 165]

1번째 훈련 폴드: [110 132]
1번째 검증 폴드: [28 33]

2번째 훈련 폴드: [110 132]
2번째 검증 폴드: [28 33]

3번째 훈련 폴드: [110 132]
3번째 검증 폴드: [28 33]

4번째 훈련 폴드: [111 132]
4번째 검증 폴드: [27 33]

5번째 훈련 폴드: [111 132]
5번째 검증 폴드: [27 33]



In [11]:
kfold

StratifiedKFold(n_splits=5, random_state=2, shuffle=True)

In [12]:
scores = cross_val_score(model, X, y, cv=kfold)
print('accuracy:', np.round(scores, 2))
print('accuracy mean:', np.round(scores.mean(), 2))

accuracy: [0.75 0.75 0.75 0.82 0.87]
accuracy mean: 0.79


In [13]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [41]:
def grid_search(params, random=False):
  xgb = XGBClassifier(booster='gbtree', objective='binary:logistic',
                      random_state=2, verbosity=0, use_label_encoder=False)
  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

  if random:
    grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iter=20,
                              n_jobs=-1, random_state=2)
  else:
    grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
  
  grid.fit(X, y)
  best_params = grid.best_params_
  print('best parameters:', best_params)
  best_score = grid.best_score_
  print('best score:', best_score)

In [15]:
grid_search(params={'n_estimators':[100, 200, 400, 800]})

best parameters: {'n_estimators': 100}
best score: 0.7891256830601093


In [16]:
grid_search(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

best parameters: {'learning_rate': 0.05}
best score: 0.8056284153005464


In [17]:
grid_search(params={'max_depth':[2, 3, 5, 6, 8]})

best parameters: {'max_depth': 2}
best score: 0.8056830601092896


In [18]:
grid_search(params={'gamma':[0, 0.01, 0.1, 0.5, 1, 2]})

best parameters: {'gamma': 1}
best score: 0.8022404371584699


In [19]:
grid_search(params={'min_child_weight':[1, 2, 3, 4, 5]})

best parameters: {'min_child_weight': 2}
best score: 0.8087978142076502


In [20]:
grid_search(params={'subsample':[0.5, 0.7, 0.8, 0.9, 1]})

best parameters: {'subsample': 0.7}
best score: 0.8056830601092896


In [21]:
grid_search(params={'colsample_bytree':[0.5, 0.7, 0.8, 0.9, 1]})

best parameters: {'colsample_bytree': 0.5}
best score: 0.7989071038251366


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [23]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')

In [24]:
eval_set = [(X_test, y_test)]

In [25]:
eval_metric='error'

In [26]:
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set)

[0]	validation_0-error:0.171053
[1]	validation_0-error:0.118421
[2]	validation_0-error:0.157895
[3]	validation_0-error:0.184211
[4]	validation_0-error:0.157895
[5]	validation_0-error:0.171053
[6]	validation_0-error:0.157895
[7]	validation_0-error:0.144737
[8]	validation_0-error:0.144737
[9]	validation_0-error:0.144737
[10]	validation_0-error:0.131579
[11]	validation_0-error:0.144737
[12]	validation_0-error:0.144737
[13]	validation_0-error:0.144737
[14]	validation_0-error:0.144737
[15]	validation_0-error:0.144737
[16]	validation_0-error:0.157895
[17]	validation_0-error:0.131579
[18]	validation_0-error:0.144737
[19]	validation_0-error:0.144737
[20]	validation_0-error:0.144737
[21]	validation_0-error:0.144737
[22]	validation_0-error:0.144737
[23]	validation_0-error:0.144737
[24]	validation_0-error:0.144737
[25]	validation_0-error:0.144737
[26]	validation_0-error:0.131579
[27]	validation_0-error:0.131579
[28]	validation_0-error:0.131579
[29]	validation_0-error:0.131579
[30]	validation_0-er

XGBClassifier()

In [27]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)

accuracy: 0.8552631578947368


In [28]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')
eval_set = [(X_test, y_test)]
eval_metric='error'
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set,
          early_stopping_rounds=10, verbose=True)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)

[0]	validation_0-error:0.171053
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.118421
[2]	validation_0-error:0.157895
[3]	validation_0-error:0.184211
[4]	validation_0-error:0.157895
[5]	validation_0-error:0.171053
[6]	validation_0-error:0.157895
[7]	validation_0-error:0.144737
[8]	validation_0-error:0.144737
[9]	validation_0-error:0.144737
[10]	validation_0-error:0.131579
[11]	validation_0-error:0.144737
Stopping. Best iteration:
[1]	validation_0-error:0.118421

accuracy: 0.881578947368421


In [29]:
model = XGBClassifier(n_estimators=5000)
eval_set = [(X_test, y_test)]
eval_metric='error'
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set,
          early_stopping_rounds=100, verbose=True)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)

[0]	validation_0-error:0.171053
Will train until validation_0-error hasn't improved in 100 rounds.
[1]	validation_0-error:0.118421
[2]	validation_0-error:0.157895
[3]	validation_0-error:0.184211
[4]	validation_0-error:0.157895
[5]	validation_0-error:0.171053
[6]	validation_0-error:0.157895
[7]	validation_0-error:0.144737
[8]	validation_0-error:0.144737
[9]	validation_0-error:0.144737
[10]	validation_0-error:0.131579
[11]	validation_0-error:0.144737
[12]	validation_0-error:0.144737
[13]	validation_0-error:0.144737
[14]	validation_0-error:0.144737
[15]	validation_0-error:0.144737
[16]	validation_0-error:0.157895
[17]	validation_0-error:0.131579
[18]	validation_0-error:0.144737
[19]	validation_0-error:0.144737
[20]	validation_0-error:0.144737
[21]	validation_0-error:0.144737
[22]	validation_0-error:0.144737
[23]	validation_0-error:0.144737
[24]	validation_0-error:0.144737
[25]	validation_0-error:0.144737
[26]	validation_0-error:0.131579
[27]	validation_0-error:0.131579
[28]	validation_0-e

In [30]:
grid_search(params={'n_estimators':[2, 25, 50, 75, 100]})

best parameters: {'n_estimators': 75}
best score: 0.8022950819672131


In [32]:
grid_search(params={'max_depth':[1, 2, 3, 4, 6, 7, 8],
                    'n_estimators':[75]})

best parameters: {'max_depth': 1, 'n_estimators': 75}
best score: 0.8221857923497268


In [33]:
grid_search(params={'max_depth':[1, 2, 3, 4, 6, 7, 8],
                    'n_estimators':[2, 50, 75, 100]})

best parameters: {'max_depth': 1, 'n_estimators': 100}
best score: 0.8254644808743169


In [36]:
grid_search(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5], 
                    'max_depth':[1],
                    'n_estimators':[20, 50, 75, 100]})

best parameters: {'learning_rate': 0.3, 'max_depth': 1, 'n_estimators': 50}
best score: 0.838688524590164


In [37]:
grid_search(params={'min_child_weight':[1, 2, 3, 4, 5], 
                    'max_depth':[1],
                    'n_estimators':[50]})

best parameters: {'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 50}
best score: 0.8090163934426229


In [38]:
grid_search(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'max_depth':[1],
                    'n_estimators':[50]})

best parameters: {'max_depth': 1, 'n_estimators': 50, 'subsample': 0.8}
best score: 0.8253551912568305


In [39]:
grid_search(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'min_child_weight':[1, 2, 3, 4, 5],
                    'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5],
                    'max_depth':[1, 2, 3, 4, 5],
                    'n_estimators':[2]})

best parameters: {'learning_rate': 0.5, 'max_depth': 2, 'min_child_weight': 4, 'n_estimators': 2, 'subsample': 0.9}
best score: 0.81224043715847


In [42]:
grid_search(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'min_child_weight':[1, 2, 3, 4, 5],
                    'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5],
                    'max_depth':[1, 2, 3, 4, 5, None],
                    'n_estimators':[2, 25, 50, 75, 100]}, random=True)

best parameters: {'subsample': 0.6, 'n_estimators': 25, 'min_child_weight': 4, 'max_depth': 4, 'learning_rate': 0.5}
best score: 0.8220765027322404


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py", line 732, in fit
    callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/training.py", line 216, in train
    xgb_model=xgb_model, callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/training.py", line

In [46]:
grid_search(params={'colsample_bytree':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'max_depth':[1],
                    'n_estimators':[50]})

best parameters: {'colsample_bytree': 0.5, 'max_depth': 1, 'n_estimators': 50}
best score: 0.8155737704918034


In [47]:
grid_search(params={'colsample_bylevel':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'max_depth':[1],
                    'n_estimators':[50]})

best parameters: {'colsample_bylevel': 0.5, 'max_depth': 1, 'n_estimators': 50}
best score: 0.8155737704918034


In [48]:
grid_search(params={'colsample_bynode':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'colsample_bylevel':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'colsample_bytree':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'max_depth':[1],
                    'n_estimators':[50]})

best parameters: {'colsample_bylevel': 0.5, 'colsample_bynode': 0.5, 'colsample_bytree': 0.9, 'max_depth': 1, 'n_estimators': 50}
best score: 0.8419125683060109


In [49]:
grid_search(params={'gamma':[0, 0.01, 0.05, 0.1, 0.5, 1, 2, 3],
                    'colsample_bynode':[0.5],
                    'colsample_bylevel':[0.5],
                    'colsample_bytree':[0.9], 
                    'max_depth':[1],
                    'n_estimators':[50]})

best parameters: {'colsample_bylevel': 0.5, 'colsample_bynode': 0.5, 'colsample_bytree': 0.9, 'gamma': 0, 'max_depth': 1, 'n_estimators': 50}
best score: 0.8419125683060109
